# Where to Open a Japanese Restaurant in San Diego?

## A recent study in Zagat (https://qz.com/657013/what-americans-are-willing-to-pay-for-ethnic-restaurant-food-reveals-some-pretty-deep-prejudices/) demonstrated that Japanese food is the second most expensive cultural food that American's will spend money on.  Due to this, when deciding where to open a new Japanese restaurant, restauranteers should look to zip codes that are above the median household income of the US, as well as a low number of existing Japanese restaurants

### I set out to determine which zip codes in San Diego offer a low number of competing locations within zip codes above the median household income for the US